<a href="https://colab.research.google.com/github/bmnds/uea-data-science-03-statistics/blob/main/%5BEstat%C3%ADstica04%5D_Trabalho_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho Final da Disciplina de Estatística do Curso de Pós-Graduação em Ciência de Dados da UEA-AM
**Grupo:** Alberto, Bruno, Lelson e Levi

## O Trabalho
* Definir um tema e os objetivos a serem analisados 
* Escolheru m dataframe com no mínimo cinco campos e 25 linhas, dos quais pelo menos dois devem ser numéricos
* Realizar um levantamento amostral atendendo a todos os critérios de avaliação

## A Avaliação
1. [ ] Construir tabelas e gráficos apropriados para cada variável do data frame
2. [ ] Calcular a média, mediana, moda, desvio-padrão, coeficiente de variação
e simetria das variáveis quantitativas
3. [ ] Analisar a normalidade dos dados numéricos e se existe a presença de
outliers
4. [ ] Comparar as variáveis quantitativas e descrever qual é mais homogênea
5. [ ] Calcular os respectivos Intervalos de Confiança ao nível de 95% para cada
campo do data frame
6. [ ] Comparar a média de um campo numérico em relação a um campo
categórico e responder se existe diferença estatística ao nível de 5% de significância
7. [ ] Realizar um cruzamento de dois campos categóricos e responder se existe
diferença estatística ao nível de 5% de significância.


In [1]:
# Pacotes necessários
from six.moves import urllib
from scipy import stats
from scipy.stats import binom, nbinom, poisson, uniform, expon, norm 
import pandas as pd
import statsmodels.stats.proportion as smp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
import statistics as st

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Mudança da semente aleatória randômica para manter os resultados em várias execuções
np.random.seed(20201109)

# Dataset de Ocorrências Aeronáuticas da Aviação Civil Brasileira
Dados fornecidos pelo [CENIPA](https://dados.gov.br/dataset/ocorrencias-aeronauticas-da-aviacao-civil-brasileira)

In [3]:
# Download do dataset
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jHYzLvKBRcoBWMbDgKqsU969sep-N8sq','cenipa.tar.gz')
!tar -zxvf cenipa.tar.gz
!rm cenipa.tar.gz

cenipa/
cenipa/aeronave.csv
cenipa/fator_contribuinte.csv
cenipa/ocorrencia.csv
cenipa/recomendacao.csv


In [4]:
bd_aeronave = pd.read_csv('cenipa/aeronave.csv',sep=';',decimal='.',encoding='iso-8859-1')
bd_aeronave.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   codigo_ocorrencia2           5307 non-null   int64  
 1   aeronave_matricula           5307 non-null   object 
 2   aeronave_operador_categoria  5307 non-null   object 
 3   aeronave_tipo_veiculo        5307 non-null   object 
 4   aeronave_fabricante          5307 non-null   object 
 5   aeronave_modelo              5307 non-null   object 
 6   aeronave_tipo_icao           5307 non-null   object 
 7   aeronave_motor_tipo          5307 non-null   object 
 8   aeronave_motor_quantidade    5307 non-null   object 
 9   aeronave_pmd                 5307 non-null   int64  
 10  aeronave_pmd_categoria       5307 non-null   int64  
 11  aeronave_assentos            5182 non-null   float64
 12  aeronave_ano_fabricacao      5189 non-null   float64
 13  aeronave_pais_fabr

In [5]:
bd_fator_contribuinte = pd.read_csv('cenipa/fator_contribuinte.csv',sep=';',decimal='.',encoding='iso-8859-1')
bd_fator_contribuinte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3717 entries, 0 to 3716
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   codigo_ocorrencia3   3717 non-null   int64 
 1   fator_nome           3717 non-null   object
 2   fator_aspecto        3717 non-null   object
 3   fator_condicionante  3717 non-null   object
 4   fator_area           3717 non-null   object
dtypes: int64(1), object(4)
memory usage: 145.3+ KB


In [6]:
bd_ocorrencia = pd.read_csv('cenipa/ocorrencia.csv',sep=';',decimal='.',encoding='iso-8859-1')
bd_ocorrencia.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5245 entries, 0 to 5244
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   codigo_ocorrencia               5245 non-null   int64 
 1   codigo_ocorrencia1              5245 non-null   int64 
 2   codigo_ocorrencia2              5245 non-null   int64 
 3   codigo_ocorrencia3              5245 non-null   int64 
 4   codigo_ocorrencia4              5245 non-null   int64 
 5   ocorrencia_classificacao        5245 non-null   object
 6   ocorrencia_latitude             3549 non-null   object
 7   ocorrencia_longitude            3548 non-null   object
 8   ocorrencia_cidade               5245 non-null   object
 9   ocorrencia_uf                   5245 non-null   object
 10  ocorrencia_pais                 5245 non-null   object
 11  ocorrencia_aerodromo            5245 non-null   object
 12  ocorrencia_dia                  5245 non-null   

In [7]:
bd_recomendacao = pd.read_csv('cenipa/recomendacao.csv',sep=';',decimal='.',encoding='iso-8859-1')
bd_recomendacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8241 entries, 0 to 8240
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   codigo_ocorrencia       8226 non-null   float64
 1   rec_numero              8241 non-null   object 
 2   rec_dia_assinatura      8232 non-null   object 
 3   rec_dia_encaminhamento  2636 non-null   object 
 4   rec_dia_feedback        5499 non-null   object 
 5   rec_status              8241 non-null   object 
 6   rec_dest_sigla          8241 non-null   object 
 7   rec_dest_nome           8241 non-null   object 
 8   rec_conteudo            8240 non-null   object 
dtypes: float64(1), object(8)
memory usage: 579.6+ KB


In [11]:
bd_recomendacao.head()

,codigo_ocorrencia,rec_numero,rec_dia_assinatura,rec_dia_encaminhamento,rec_dia_feedback,rec_status,rec_dest_sigla,rec_dest_nome,rec_conteudo
0,31379.0,047/D/10,2010-03-19,NaN,2011-01-21,***,DNA-AR,DIRECCIÃN NACIONAL DE AERONAVEGABILIDAD (DNA)...,Avaliar junto Ã AgÃªncia Nacional de AviaÃ§Ã£...
1,31379.0,047/D/10,2010-03-19,2010-05-18,2011-01-21,NÃO CUMPRIDA,DNA-AR,DIRECCIÃN NACIONAL DE AERONAVEGABILIDAD (DNA)...,Avaliar junto Ã AgÃªncia Nacional de A...
2,11281.0,074/A/00,2000-08-17,NaN,2001-10-31,***,3Âº ETA,TERCEIRO ESQUADRÃO DE TRANSPORTE AÃREO,"Divulgar, de imediato, aos seus setor..."
3,11281.0,073/A/00,2000-08-17,NaN,2001-10-31,***,3Âº ETA,TERCEIRO ESQUADRÃO DE TRANSPORTE AÃREO,"Divulgar, de imediato, para o seu qua..."
4,31855.0,079/A/08,2008-04-18,2008-04-18,2008-10-17,***,3/8Âº GAV,TERCEIRO DO OITAVO GRUPO DE AVIAÃÃO,Verificar a existÃªncia e as perfeitas condiÃ§...


In [12]:
bd_ocorrencia.head()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,39115,39115,39115,39115,39115,ACIDENTE,-13.3805555556,-44.6172222222,CORRENTINA,BA,BRASIL,SNTY,07/02/2010,17:40:00,SIM,FINALIZADA,A-031/CENIPA/2011,SIM,2011-07-21,2,1,NÃO
1,39155,39155,39155,39155,39155,INCIDENTE,NaN,NaN,BELO HORIZONTE,MG,BRASIL,****,05/02/2010,12:55:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,39156,39156,39156,39156,39156,INCIDENTE GRAVE,-15.2402777778,-59.3541666667,CASCAVEL,PR,BRASIL,SBCA,10/01/2010,23:15:00,SIM,FINALIZADA,I-004/CENIPA/2011,SIM,2011-06-30,2,1,NÃO
3,39158,39158,39158,39158,39158,INCIDENTE,***,***,BELÉM,PA,BRASIL,****,28/01/2010,16:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
4,39176,39176,39176,39176,39176,INCIDENTE,NaN,NaN,SÃO LUÍS,MA,BRASIL,****,05/02/2010,09:30:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
